In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from pyquery import PyQuery as pq
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
from pyquery import PyQuery as pq
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options 
import re
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import TimeoutException

In [ ]:
from Ekata import *
from iproxy import *
from Dossier import *
from Good_Spend import *
from diff_month import *

In [ ]:
edge_options = Options()
user_data_dir = ' '
# Here you set the path of the profile ending with User Data not the profile folder
edge_options.add_argument(user_data_dir)

# Here you specify the actual profile folder    
edge_options.add_argument("profile-directory=profile 1")

edge_options.binary_location = r"C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe"
driver = webdriver.Edge(options = edge_options)
    
# Github credentials
username = ""
password = ""

# head to Jarvis login page
driver.get("https://jarvisjx.cp.microsoft.com")

# wait login window
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tilesHolder"]/div[1]/div/div')))
# choose account 
driver.find_element(By.XPATH, '//*[@id="tilesHolder"]/div[1]/div/div').click()

# choose consumer 
driver.find_element(By.XPATH, '//*[@id="summaryNav"]/li[5]').click()

# wait consumer window
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="tenantData-1"]/table/thead/tr/th[1]')))

# web-physical
driver.find_element(By.XPATH, '//*[@id="qSelChk100002"]').click()

In [ ]:
# scroll down
driver.find_element(By.XPATH, '//*[@id="qSelChk100002"]').send_keys(Keys.PAGE_DOWN)

In [ ]:
ekata_driver = Ekata_login()

In [ ]:
ip_proxy_login = ip_proxy_login()

In [ ]:
# Dossier_login = Dossier_login(driver)

In [ ]:
# switch to the next window
driver.switch_to.window(driver.window_handles[0])

In [ ]:
# click Review 
driver.find_element(By.XPATH, '//*[@id="fltrRvwBtn"]').click()

In [ ]:
# switch to the next window
driver.switch_to.window(driver.window_handles[1])

In [ ]:
# get html 
html = driver.page_source
doc = BeautifulSoup(html)

In [ ]:
# msa date
msa = doc.find('span', class_='paren msaCreateDate').text.replace(',', '')
msa = parser.parse(msa)

# pi date
pi_date = doc.find('span', class_='paren piAddDate').text.replace(',', '')
pi_date = parser.parse(pi_date)

# email
email = doc.find('u', class_='email').text

# Account address
acc_address = doc.find('dt', class_='').find_next_siblings('dd')[1].text

# puid 
puid = doc.find('dt', class_='').find_next_siblings('dd')[4].text

# CH 
ch = doc.find('dd', attrs={'data-tag' : 'PaymentInstrument-PiFullName'}).text
ch_firstname = ch.split(' ')[1]
ch_surname = ch.split(' ')[-2]

In [ ]:
if driver.find_element(By.XPATH, '//*[@id="paymentInstrument"]/div[1]/div/dl/dd[5]').text == 'paypal':
    # pi address
    pi_address = driver.find_element(By.XPATH, '//*[@id="paymentInstrument"]/div[1]/div/dl/dd[2]').text
else:
    # pi address
    pi_address = driver.find_element(By.XPATH, '//*[@id="paymentInstrument"]/div[1]/div/dl/dd[2]').text
    # bin country
    bin_country = doc.find('dd', attrs={'data-tag' : 'PaymentInstrument-BinCountryCode'}).text


In [ ]:
# items
items = doc.find('span', attrs={'class' : 'desc'}).text

# rec name
rec_name = doc.find('dd', attrs={'data-tag' : 'Transaction-ShippingName'}).text
rec_firstname = rec_name.split(' ')[1]
rec_surname = rec_name.split(' ')[-2]

# shipping address
sa = doc.find('dd', attrs={'data-tag' : 'Transaction-FullShippingAddress'}).text

# shipping phone
sp = driver.find_element(By.XPATH, '//*[@id="transactionDetails"]/div[1]/div/dl/dd[4]').text

# PUID
tracking_guid = driver.find_element(By.XPATH, '//*[@id="transactionDetails"]/div[1]/div/dl/dd[15]').text

# ip address
ip = doc.find('dd', attrs={'data-tag' : 'IPActivity-TrueIp'}).text
ip = ip.strip(' ')

# chargeback sum
cb = driver.find_element(By.XPATH, '//*[@id="BillingHistory"]/div[1]/div/dl/dd[4]').text.strip('$')
cb = float(cb)

# good_spend
Jarvis_good_spend= driver.find_element(By.XPATH, '//*[@id="BillingHistory"]/div[1]/div/dl/dd[7]').text.strip('$')
Jarvis_good_spend = float(Jarvis_good_spend)

In [ ]:
# links

# True ip 
try:
    true_ip_user= driver.find_element(By.XPATH, '//*[@id="BrowserIPAddress"]/header/ul/li[1]').text
    true_ip_rejected = driver.find_element(By.XPATH, '//*[@id="BrowserIPAddress"]/header/ul/li[2]').text
    true_ip_approve = driver.find_element(By.XPATH, '//*[@id="BrowserIPAddress"]/header/ul/li[3]').text
except NoSuchElementException:
    true_ip_user = None
    
# PUID -> addpi event
try:
    add_pi_user =  driver.find_element(By.XPATH, '//*[@id="UserId"]/header/ul/li[1]').text
    add_pi_rejected =  driver.find_element(By.XPATH, '//*[@id="UserId"]/header/ul/li[2]').text
    add_pi_approve =  driver.find_element(By.XPATH, '//*[@id="UserId"]/header/ul/li[3]').text
except NoSuchElementException:
    add_pi_user = None
    
# PI Hash 
try:
    pi_hash_user = driver.find_element(By.XPATH, '//*[@id="CTPPIHASHID"]/header/ul/li[1]').text
    pi_hash_rejected = driver.find_element(By.XPATH, '//*[@id="CTPPIHASHID"]/header/ul/li[2]').text
    pi_hash_approve = driver.find_element(By.XPATH, '//*[@id="CTPPIHASHID"]/header/ul/li[3]').text
except NoSuchElementException:
    pi_hash_user = None
    
# Bing map address
try:
    Bing_user = driver.find_element(By.XPATH, '//*[@id="s_BingMap_SAFormattedAddress"]/header/ul/li[1]').text
    Bing_rejected = driver.find_element(By.XPATH, '//*[@id="s_BingMap_SAFormattedAddress"]/header/ul/li[1]').text
    Bing_approve = driver.find_element(By.XPATH, '//*[@id="s_BingMap_SAFormattedAddress"]/header/ul/li[1]').text
except NoSuchElementException:
    Bing_user = None


In [ ]:
# type items 
driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(items)
driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check msa
if diff_month(datetime.date(datetime.now()), msa) >= 3:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('EST MSA')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('NEW MSA')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check pi_date
if diff_month(datetime.date(datetime.now()), pi_date) >= 3:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('est pi')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('new pi')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check if API match 
if fuzz.partial_ratio(acc_address, pi_address) >= 75:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('API Match')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('API Mismatch')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check sa with ch
ekata_drive, first_name, family_name = Ekata_address(driver=ekata_driver, address=sa.split(',')[:-2], state=sa.split(',')[-2], country=sa.split(',')[-1], 
              first_name=ch_firstname, family_name=ch_surname)

if first_name==True and family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sa linked to ch')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sa linked to ch family name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif first_name==True:
    driver.find_element(Bye.XPATH, '//*[@id="notes"]/input').send_keys('sa linked to ch first name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sa not linked to ch')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check sa with rec
ekata_drive, first_name, family_name = Ekata_address(ekata_driver, address=sa.split(',')[:-2], state=sa.split(',')[-2], country=sa.split(',')[-1], 
              first_name=rec_firstname, family_name=rec_surname)

if first_name==True and family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sa linked to rec')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sa linked to rec family name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif first_name==True:
    driver.find_element(Bye.XPATH, '//*[@id="notes"]/input').send_keys('sa linked to rec first name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sa not linked rec')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check sp with ch
ekata_drive, first_name, family_name = Ekata_phone(ekata_driver, country=sa.split(',')[-1].strip().upper()+' ', phone=sp,
              first_name=ch_firstname, family_name=ch_surname)

if first_name==True and family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp linked to ch')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp linked to ch family name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif first_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp linked to ch first name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp not linked to ch')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check sp with rec
ekata_drive, first_name, family_name = Ekata_phone(ekata_driver, country=sa.split(',')[-1].strip().upper()+' ', phone=sp,
              first_name=rec_firstname, family_name=rec_surname)

if first_name==True and family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp linked to rec')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif family_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp linked to rec family name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif first_name==True:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp linked to rec first name')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('sp not linked to rec')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# check ip
location, conn_type, register_org, ekata_driver = Ekata_IP(ekata_driver, ip)
if_proxy = ip_proxy_check(ip_proxy_login, ip)[1]

# check ip cellular or not
if conn_type == 'Cellular':
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('cellular ip')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
# check if proxy ip
elif if_proxy:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('proxy ip')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
    print(ip_proxy_check(ip_proxy_login, ip)[1])
# check if geoip match
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('no proxy')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
    
    if location.split(',')[0] in (pi_address or sa) :
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('geoip match')
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
        
    else:
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('geoip mismatch')
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)

In [ ]:
# good spend
if Jarvis_good_spend > 0:
    cb_ratio, good_spend, bad_spend = pay_load(driver=driver, risk_guid=tracking_guid)
    driver.switch_to.window(driver.window_handles[1])
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('good spend'+str(good_spend))
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
   
    if bad_spend == 0:
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('no bad spend').send_keys('no cb')
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
      
    else:
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('ratio').send_keys(cb_ratio)
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('bad_spend').send_keys(bad_spend)
        driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
        if cb_ratio < 1:
            driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('under')
            driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)
elif cb > 0:
    cb_ratio, good_spend, bad_spend = pay_load(driver=driver, risk_guid=tracking_guid)
    driver.switch_to.window(driver.window_handles[1])
    print(cb_ratio)
    print(bad_spend)
else:
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys('no good / bad spend')
    driver.find_element(By.XPATH, '//*[@id="notes"]/input').send_keys(Keys.ENTER)


In [ ]:
# approve
driver.find_element(By.XPATH, '//*[@id="accept"]').click()

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="acceptMenu"]/ul/li[5]')))

driver.find_element(By.XPATH, '//*[@id="acceptMenu"]/ul/li[5]').click()
